# Load Data

In [1]:
import pandas as pd
import os

In [2]:
all_data = []
if os.getcwd().split("\\")[len(os.getcwd().split("\\")) - 1] != "data":
    os.chdir('./data')

for f in os.listdir():
    # print(f)
    part_id = f.split("_")[1].split(".")[0]
    # print(part_id)
    df = pd.read_csv(f, sep=";")
    df["part_id"] = part_id
    all_data.append(df)

In [3]:
print(len(all_data))
all_data[0].head()

60


,timestamp,active,xPos,yPos,TargetX,StartX,TunnelAmplitude,TunnelWidth,YDistanceToTop,YDistanceToBottom,part_id
0,1652861652030,0,0,0,79.5,1286.5,1207,57,-284.0,341.0,12299398
1,1652861652056,0,0,0,79.5,1286.5,1207,57,-284.0,341.0,12299398
2,1652861652116,0,676,593,79.5,1286.5,1207,57,237.5,-180.5,12299398
3,1652861652128,0,676,593,79.5,1286.5,1207,57,237.5,-180.5,12299398
4,1652861652140,0,676,593,79.5,1286.5,1207,57,237.5,-180.5,12299398


# Preprocess Data

## Normalize by Participants Displaysize

In [3]:
import copy

In [4]:
def normalize_by_screen(datapoint, screenwidth, screenheight):
    normalized = copy.deepcopy(datapoint)
    normalized["xPos"] /= screenwidth
    normalized["TargetX"] /= screenwidth
    normalized["StartX"] /= screenwidth
    normalized["TunnelAmplitude"] /= screenwidth
    normalized["yPos"] /= screenheight
    normalized["TunnelWidth"] /= screenheight
    normalized["YDistanceToTop"] /= screenheight
    normalized["YDistanceToBottom"] /= screenheight
    return normalized

In [5]:
def get_participants_screensize(datapoint):
    x_res = -1
    y_res = -1
    # print(datapoint)
    curr_part_id = datapoint["part_id"][0]
    # print(part_id)
    for f in os.listdir():
        curr_id = f.split("_")[1].split(".")[0]
        if curr_id == curr_part_id:
            demographic = pd.read_csv(f, sep=";")
            real_res_string = demographic["Real Res"][0]
            real_res_string = real_res_string.replace("px", "")
            x_res = real_res_string.split(" x ")[0]
            y_res = real_res_string.split(" x ")[1]
    return int(x_res), int(y_res)

In [6]:
def normalize_screensize(dataset):
    if os.getcwd().split("\\")[len(os.getcwd().split("\\")) - 1] != "demographic_data":
        os.chdir("../demographic_data")
    for i, datapoint in enumerate(dataset):
        screenwidth, screenheight = get_participants_screensize(datapoint)
        dataset[i] = normalize_by_screen(datapoint, screenwidth, screenheight)

## First Order Derivation
The goal is to make the movement independent of the positions
Therefore, xPos and yPos are converted to speeds by taking the first derivation

In [35]:
def get_data_with_derivate(datapoint):
    diffSeries = datapoint["xPos"].diff(periods=1)
    datapoint.insert(3, "xSpeed", diffSeries)
    diffSeries = datapoint["yPos"].diff(periods=1)
    datapoint.insert(5, "ySpeed", diffSeries)

## Umwandeln in Array

In [7]:
import numpy as np

In [8]:
def get_np_arrays(data):
    data_array = []
    for curr in data:
        removed = copy.deepcopy(curr)
        removed = removed.drop('timestamp', axis=1)
        removed = removed.drop('xPos', axis=1)
        removed = removed.drop('yPos', axis=1)
        removed = removed.drop('part_id', axis=1)
        data_array.append(removed.to_numpy(copy=True, dtype='float32'))
    return data_array

## Split into sections of n samples

In [9]:
def append_samples_of_length_n(participant_data, n, samples, labels, label_distance):
    for i in range(participant_data.shape[0] - (n+label_distance)):
        samples.append(copy.deepcopy(participant_data[i:i+n]))
        sum_to_label = np.array([0,0], dtype='float64')
        for j in range (i+n+1, i+n+label_distance+1):
            sum_to_label += participant_data[j][1:3]
        labels.append(copy.deepcopy(sum_to_label))
        # labels.append(copy.deepcopy(participant_data[i+n+label_distance][1:3]))

In [10]:
def get_samples_and_labels(data_array, N=10, label_distance=10):
    samples = []
    labels = []
    for participant_data in data_array:
        append_samples_of_length_n(participant_data, N, samples, labels, label_distance)
    return samples, labels

## Remove unusable samples
If in one sample all 10 xSpeeds and ySpeeds are equal the sample is removed, if between the last sample and the label the mouse isn't moved the sample is removed

In [11]:
def remove_unusable(data, labels, verbose=0):
    # print(labels)
    new_data = []
    new_labels = []
    for i, sample in enumerate(data):
        bool_array = np.all(sample == sample[0,:], axis=0)
        if bool_array[1] == False or bool_array[2] == False:
            if labels[i][0] != 0:
                new_data.append(sample)
                new_labels.append(labels[i])
        if verbose > 1 and i % 100000 == 0:
            print(f'evaluated {i}')

    if verbose > 0:
        print(f'removed {len(data) - len(new_data)} samples')

    return new_data, new_labels

## Make all Samples face the same way
The average xSpeed is calculated, if this average is negative all xSpeeds are multiplied by one to make all vectors face in the same direction

In [12]:
def make_same_orientation(data, verbose=0):
    changed = 0
    for sample in data:
        average_x_speed = np.average(sample, axis=0)[1]
        if average_x_speed < 0:
            sample[:, 1] *= -1
            if verbose > 0:
                changed += 1
    if verbose > 0:
        print(f'Changed {changed} samples')

## Transform a dataset

In [13]:
def get_transformed_data(dataset, N, label_distance, verbose):
    normalize_screensize(dataset)

    if'xSpeed' not in dataset[0].columns:
        for datapoint in dataset:
            get_data_with_derivate(datapoint)
            datapoint.drop(index=0, axis=0, inplace=True)

    data_array = get_np_arrays(dataset)

    samples, labels = get_samples_and_labels(data_array, N, label_distance)

    samples, labels = remove_unusable(samples, labels, verbose=verbose)

    make_same_orientation(samples, verbose=verbose)

    return samples, labels

## Train-Test-Split
Initially the entire dataset given in all_data are split into Test and Train group by participants, than the datasets are preprocessed in the same way

In [14]:
TRAIN_SPLIT = 0.8
TEST_SPLIT = 1.0 - TRAIN_SPLIT

In [15]:
import random as rand
SEED = 16961

In [16]:
def get_train_test_split_data(dataset):
    train_data_amount = 0.8 * len(dataset)

    train_data = []
    test_data = []

    rand.seed(SEED)
    train_indizes = rand.sample(range(len(dataset)), int(train_data_amount))
    # print(train_indizes)

    for i, sample in enumerate(dataset):
        if i in train_indizes:
            train_data.append(sample)
        else:
            test_data.append(sample)

    # print(len(train_data))
    # print(len(test_data))

    return train_data, test_data

## Normalize Trainingdata
The average column values of the training data is calculated, each value in test and training data is divided by the average of its column

In [17]:
def get_average_array(arr, axis=(0,1)):
    return np.mean(arr, axis=axis)

In [18]:
def get_normalized_arrays(train_samples, train_labels, test_samples, test_labels):
    avg_array = get_average_array(train_samples)
    avg_label_array = get_average_array(train_labels, 0)

    train_array = np.array(train_samples)
    test_array = np.array(test_samples)
    train_labels_array = np.array(train_labels)
    test_labels_array = np.array(test_labels)

    train_normalized = train_array[:, :, ] / avg_array
    train_normalized[:, : , 0] = train_array[:, :, 0]
    train_normalized[:, :, 2] = train_array[:, :, 2]

    test_normalized = test_array[:, :, ] / avg_array
    test_normalized[:, :, 0] = test_array[:, :, 0]
    test_normalized[:, :, 2] = test_array[:, :, 2]

    # train_labels_array[:, 0] /= avg_label_array[0:1] - averages are very low!

    # test_labels_array[:, 0] /= avg_label_array[0:1]

    return train_normalized, train_labels_array, test_normalized, test_labels_array

## Shuffle Arrays

In [19]:
from sklearn.utils import shuffle

In [20]:
VERBOSE = 1
LABEL_DISTANCE = 6 # 6 => 100ms

def get_preprocessed_data(N):
    train_data, test_data = get_train_test_split_data(all_data)

    train_samples, train_labels = get_transformed_data(train_data, N, LABEL_DISTANCE, 0)
    test_samples, test_labels = get_transformed_data(test_data, N, LABEL_DISTANCE, 0)

    train_samples, train_labels, test_samples, test_labels = get_normalized_arrays(train_samples, train_labels, test_samples, test_labels)

    train_samples, train_labels = shuffle(train_samples, train_labels, random_state=SEED)
    test_samples, test_labels = shuffle(test_samples, test_labels, random_state=SEED)

    return train_samples, train_labels, test_samples, test_labels

# Hyperparameter Optimization

In [21]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import optuna

In [22]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [23]:
OPTIMIZATION_SPLIT = 0.2
NUMBER_OF_TRIALS = 200
EPOCHS = 10

## Fully-Connected Feed-Forward Neural Network

Optimize Dataset - How many samples to predict next

In [34]:
def create_dataset(trial):
    N = trial.suggest_int('sequence_length', 5, 30)

    train_samples, train_labels, test_samples, test_labels = get_preprocessed_data(N)

    highest_index_train = int(train_samples.shape[0] * OPTIMIZATION_SPLIT)
    highest_index_test = int(test_samples.shape[0] * OPTIMIZATION_SPLIT)

    train_samples = train_samples.reshape(train_samples.shape[0], train_samples.shape[1] * train_samples.shape[2])
    test_samples = test_samples.reshape(test_samples.shape[0], test_samples.shape[1] * test_samples.shape[2])

    return train_samples[:highest_index_train], train_labels[:highest_index_train], test_samples[:highest_index_test], test_labels[:highest_index_test]

Optimize Model
- Number of hidden layers
- Neurons per layer
- Dropoutrate
- Activation function
- weight initialization

In [35]:
def create_model(trial, train_X):
    number_of_inputs = train_X.shape[1]

    number_of_hidden_neurons = trial.suggest_int('hidden_neurons', 5, 1000)
    number_of_hidden_layers = trial.suggest_int('hidden_layers', 1, 20)
    dropout_rate = trial.suggest_float('dropout_rate', 0., 0.5)
    activation_function = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])

    model = keras.Sequential()
    model.add(layers.Input(shape=(number_of_inputs, )))
    model.add(layers.BatchNormalization())

    for _ in range(number_of_hidden_layers):
        model.add(layers.Dense(number_of_hidden_neurons, activation=activation_function))

    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(2, activation='linear'))

    return model

Optimize Training
- Optimizer
- Loss function (?)
- Learning rate
- batch size
- Epochs (?)

**Prune bad Trials!**

In [36]:
def objective(trial):
    train_X, train_Y, test_X, test_Y = create_dataset(trial)

    model = create_model(trial, train_X)

    monitor = 'val_loss'

    callbacks = [
        # keras.callbacks.EarlyStopping(patience=10),
        optuna.integration.TFKerasPruningCallback(trial, monitor),
    ]

    # lr = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'rmsprop', 'adagrad', 'adam'])
    batch_size = trial.suggest_int('batch_size', 1, 1000)

    model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError(), metrics=['mean_absolute_percentage_error'])

    history = model.fit(train_X, train_Y, batch_size=batch_size, validation_split=0.1, verbose=1, epochs=EPOCHS, callbacks=callbacks)

    eval_results = model.evaluate(test_X, test_Y, batch_size=batch_size)

    return float(eval_results[0])

Start Optuna Optimization

In [37]:
def make_study():
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler=optuna.samplers.TPESampler())

    study.optimize(objective, n_trials=NUMBER_OF_TRIALS)

    # show results
    pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
    complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

    print('Study statistics:')
    print('   Number of finished Trials: ', len(study.trials))
    print('   Number of pruned Trials: ', len(pruned_trials))
    print('   Number of complete Trials: ', len(complete_trials))

    print('Best Trial: ')
    trial = study.best_trial

    print('  MSE: ', trial.value)

    print('   Params: ')
    for key, value in trial.params.items():
        print('   {}: {}'.format(key, value))

In [38]:
make_study()

[I 2022-06-07 12:44:07,613] A new study created in memory with name: no-name-2ac73301-e3b2-42f0-b38f-79a105f80119


Epoch 1/10
997/997 [==============================] - 34s 26ms/step - loss: 0.2936 - mean_absolute_percentage_error: 82204536.0000 - val_loss: 0.0329 - val_mean_absolute_percentage_error: 30362768.0000
Epoch 2/10
997/997 [==============================] - 27s 27ms/step - loss: 0.0250 - mean_absolute_percentage_error: 26987402.0000 - val_loss: 0.0088 - val_mean_absolute_percentage_error: 18433136.0000
Epoch 3/10
997/997 [==============================] - 26s 26ms/step - loss: 0.0069 - mean_absolute_percentage_error: 6921556.5000 - val_loss: 0.0032 - val_mean_absolute_percentage_error: 1119292.2500
Epoch 4/10
997/997 [==============================] - 25s 26ms/step - loss: 0.0064 - mean_absolute_percentage_error: 6020566.5000 - val_loss: 0.0088 - val_mean_absolute_percentage_error: 16809938.0000
Epoch 5/10
997/997 [==============================] - 25s 25ms/step - loss: 0.0060 - mean_absolute_percentage_error: 5131376.0000 - val_loss: 0.0031 - val_mean_absolute_percentage_error: 607911.8

[I 2022-06-07 12:53:58,254] Trial 0 finished with value: 0.00288063520565629 and parameters: {'sequence_length': 19, 'hidden_neurons': 278, 'hidden_layers': 16, 'dropout_rate': 0.19068054717816801, 'activation': 'tanh', 'optimizer': 'rmsprop', 'batch_size': 348}. Best is trial 0 with value: 0.00288063520565629.


Epoch 1/10
549/549 [==============================] - 5s 8ms/step - loss: 0.1567 - mean_absolute_percentage_error: 64460992.0000 - val_loss: 0.0032 - val_mean_absolute_percentage_error: 738157.0625
Epoch 2/10
549/549 [==============================] - 4s 8ms/step - loss: 0.0232 - mean_absolute_percentage_error: 24089950.0000 - val_loss: 0.0032 - val_mean_absolute_percentage_error: 536137.9375
Epoch 3/10
549/549 [==============================] - 4s 8ms/step - loss: 0.0094 - mean_absolute_percentage_error: 13671872.0000 - val_loss: 0.0032 - val_mean_absolute_percentage_error: 437018.1875
Epoch 4/10
549/549 [==============================] - 5s 8ms/step - loss: 0.0058 - mean_absolute_percentage_error: 9104158.0000 - val_loss: 0.0032 - val_mean_absolute_percentage_error: 539042.9375
Epoch 5/10
549/549 [==============================] - 4s 8ms/step - loss: 0.0045 - mean_absolute_percentage_error: 6604597.0000 - val_loss: 0.0032 - val_mean_absolute_percentage_error: 275001.1562
Epoch 6/10
5

[I 2022-06-07 12:59:04,561] Trial 1 finished with value: 0.002840838860720396 and parameters: {'sequence_length': 16, 'hidden_neurons': 243, 'hidden_layers': 6, 'dropout_rate': 0.49936005513335796, 'activation': 'sigmoid', 'optimizer': 'sgd', 'batch_size': 626}. Best is trial 1 with value: 0.002840838860720396.


...

Epoch 1/10
369/369 [==============================] - 19s 49ms/step - loss: 0.0029 - mean_absolute_percentage_error: 158947.7344 - val_loss: 0.0029 - val_mean_absolute_percentage_error: 74549.0938
Epoch 2/10
369/369 [==============================] - 18s 49ms/step - loss: 0.0029 - mean_absolute_percentage_error: 138243.4844 - val_loss: 0.0029 - val_mean_absolute_percentage_error: 80891.3438


[I 2022-06-08 02:04:45,122] Trial 99 pruned. Trial was pruned at epoch 1.


Epoch 1/10
965/965 [==============================] - 29s 25ms/step - loss: 0.0188 - mean_absolute_percentage_error: 20302514.0000 - val_loss: 0.0036 - val_mean_absolute_percentage_error: 3060629.7500
Epoch 2/10
965/965 [==============================] - 24s 25ms/step - loss: 0.0108 - mean_absolute_percentage_error: 13957844.0000 - val_loss: 0.0035 - val_mean_absolute_percentage_error: 2312040.7500


[I 2022-06-08 02:11:20,079] Trial 100 pruned. Trial was pruned at epoch 1.


Study statistics:
   Number of finished Trials:  100
   Number of pruned Trials:  60
   Number of complete Trials:  40
Best Trial: 
  MSE:  0.0028108484111726284
   Params: 
   sequence_length: 5
   hidden_neurons: 154
   hidden_layers: 17
   dropout_rate: 0.2295818303474188
   activation: tanh
   optimizer: adam
   batch_size: 382


## RNN

In [31]:
def create_rnn_dataset(trial):
    N = trial.suggest_int('sequence_length', 5, 30)

    train_samples, train_labels, test_samples, test_labels = get_preprocessed_data(N)

    highest_index_train = int(train_samples.shape[0] * OPTIMIZATION_SPLIT)
    highest_index_test = int(test_samples.shape[0] * OPTIMIZATION_SPLIT)

    return train_samples[:highest_index_train], train_labels[:highest_index_train], test_samples[:highest_index_test], test_labels[:highest_index_test]

In [36]:
def create_rnn_model(trial, train_X):
    number_of_inputs = train_X.shape[1]

    number_of_hidden_neurons = trial.suggest_int('hidden_neurons', 5, 300)
    number_of_hidden_layers = trial.suggest_int('hidden_layers', 1, 5)
    dropout_rate = trial.suggest_float('dropout_rate', 0., 0.5)
    activation_function = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
    normalize = trial.suggest_categorical('normalize', [True, False])

    model = keras.Sequential()
    if number_of_hidden_layers > 1:
        model.add(layers.SimpleRNN(number_of_hidden_neurons, activation=activation_function, input_shape=(number_of_inputs, 9), return_sequences=True))
    else:
        model.add(layers.SimpleRNN(number_of_hidden_neurons, activation=activation_function, input_shape=(number_of_inputs, 9), return_sequences=False))
        if normalize:
            model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))
        model.add(layers.Dense(2, activation='linear'))
        return model


    if normalize:
        model.add(layers.BatchNormalization())

    for _ in range(number_of_hidden_layers-2):
        model.add(layers.SimpleRNN(number_of_hidden_neurons, activation=activation_function, return_sequences=True))
        if normalize:
            model.add(layers.BatchNormalization())

    model.add(layers.SimpleRNN(number_of_hidden_neurons, activation=activation_function, return_sequences=False))
    if normalize:
        model.add(layers.BatchNormalization())

    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(2, activation='linear'))

    return model

In [37]:
def rnn_objective(trial):
    train_X, train_Y, test_X, test_Y = create_rnn_dataset(trial)

    model = create_rnn_model(trial, train_X)

    monitor = 'val_loss'

    callbacks = [
        # keras.callbacks.EarlyStopping(patience=10),
        optuna.integration.TFKerasPruningCallback(trial, monitor),
    ]

    # lr = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'rmsprop', 'adagrad', 'adam'])
    batch_size = trial.suggest_int('batch_size', 32, 200)

    model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError(), metrics=['mean_absolute_percentage_error'])

    history = model.fit(train_X, train_Y, batch_size=batch_size, validation_split=0.1, verbose=1, epochs=EPOCHS, callbacks=callbacks)

    eval_results = model.evaluate(test_X, test_Y, batch_size=batch_size)
    print(f'Result: {float(eval_results[0])}')

    return float(eval_results[0])

In [40]:
def make_rnn_study():
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler=optuna.samplers.TPESampler())

    study.optimize(rnn_objective, n_trials=100)

    # show results
    pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
    complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

    print('Study statistics:')
    print('   Number of finished Trials: ', len(study.trials))
    print('   Number of pruned Trials: ', len(pruned_trials))
    print('   Number of complete Trials: ', len(complete_trials))

    print('Best Trial: ')
    trial = study.best_trial

    print('  MSE: ', trial.value)

    print('   Params: ')
    for key, value in trial.params.items():
        print('   {}: {}'.format(key, value))

In [ ]:
make_rnn_study()

[I 2022-06-10 11:09:36,915] A new study created in memory with name: no-name-bdbabcf1-8e2b-483f-8f3c-0e0a052ecda2


Epoch 1/10
4526/4526 [==============================] - 1214s 267ms/step - loss: 0.0482 - mean_absolute_percentage_error: 9380647.0000 - val_loss: 0.0031 - val_mean_absolute_percentage_error: 479944.7500
Epoch 2/10
4526/4526 [==============================] - 1198s 265ms/step - loss: 0.0029 - mean_absolute_percentage_error: 527095.2500 - val_loss: 0.0031 - val_mean_absolute_percentage_error: 608616.5000
Epoch 3/10
4526/4526 [==============================] - 1207s 267ms/step - loss: 0.0029 - mean_absolute_percentage_error: 406984.1250 - val_loss: 0.0031 - val_mean_absolute_percentage_error: 194492.4375
Epoch 4/10
4526/4526 [==============================] - 1209s 267ms/step - loss: 0.0029 - mean_absolute_percentage_error: 430779.2500 - val_loss: 0.0031 - val_mean_absolute_percentage_error: 306917.4375
Epoch 5/10
4526/4526 [==============================] - 1214s 268ms/step - loss: 0.0029 - mean_absolute_percentage_error: 412074.6250 - val_loss: 0.0031 - val_mean_absolute_percentage_err

[I 2022-06-10 14:39:29,666] Trial 0 finished with value: 0.0028397408314049244 and parameters: {'sequence_length': 28, 'hidden_neurons': 58, 'hidden_layers': 3, 'dropout_rate': 0.440584636751394, 'activation': 'tanh', 'normalize': True, 'optimizer': 'rmsprop', 'batch_size': 78}. Best is trial 0 with value: 0.0028397408314049244.


Epoch 1/10
2103/2103 [==============================] - 447s 211ms/step - loss: 0.1280 - mean_absolute_percentage_error: 20405672.0000 - val_loss: 0.3318 - val_mean_absolute_percentage_error: 164742160.0000
Epoch 2/10
2103/2103 [==============================] - 440s 209ms/step - loss: 0.0030 - mean_absolute_percentage_error: 391534.8125 - val_loss: 0.0110 - val_mean_absolute_percentage_error: 22504752.0000
Epoch 3/10
2103/2103 [==============================] - 425s 202ms/step - loss: 0.0030 - mean_absolute_percentage_error: 458044.5000 - val_loss: 0.0117 - val_mean_absolute_percentage_error: 28981610.0000
Epoch 4/10
2103/2103 [==============================] - 416s 198ms/step - loss: 0.0030 - mean_absolute_percentage_error: 465770.3750 - val_loss: 0.0081 - val_mean_absolute_percentage_error: 18184890.0000
Epoch 5/10
2103/2103 [==============================] - 416s 198ms/step - loss: 0.0030 - mean_absolute_percentage_error: 483925.4062 - val_loss: 0.0051 - val_mean_absolute_percentag

[I 2022-06-10 15:54:35,682] Trial 1 finished with value: 0.06824205815792084 and parameters: {'sequence_length': 14, 'hidden_neurons': 175, 'hidden_layers': 4, 'dropout_rate': 0.46607273679449474, 'activation': 'sigmoid', 'normalize': True, 'optimizer': 'rmsprop', 'batch_size': 162}. Best is trial 0 with value: 0.0028397408314049244.


...

Epoch 1/10
5984/5984 [==============================] - 2938s 488ms/step - loss: 0.0139 - mean_absolute_percentage_error: 13480805.0000 - val_loss: 0.0048 - val_mean_absolute_percentage_error: 3961254.0000
Epoch 2/10
5984/5984 [==============================] - 2921s 488ms/step - loss: 0.0061 - mean_absolute_percentage_error: 7843779.5000 - val_loss: 0.0035 - val_mean_absolute_percentage_error: 3239025.0000


[I 2022-06-11 09:03:50,007] Trial 99 pruned. Trial was pruned at epoch 1.


Epoch 1/10
2751/2751 [==============================] - 495s 177ms/step - loss: 0.0644 - mean_absolute_percentage_error: 14688581.0000 - val_loss: 0.0033 - val_mean_absolute_percentage_error: 1002182.0000
Epoch 2/10
2751/2751 [==============================] - 488s 177ms/step - loss: 0.0031 - mean_absolute_percentage_error: 691031.0625 - val_loss: 0.0032 - val_mean_absolute_percentage_error: 1077751.5000


[I 2022-06-11 09:26:15,338] Trial 100 pruned. Trial was pruned at epoch 1.


Study statistics:
   Number of finished Trials:  100
   Number of pruned Trials:  76
   Number of complete Trials:  24
Best Trial: 
  MSE:  0.09282874974024
   Params: 
   sequence_length: 7
   hidden_layers: 1
   number_of_neurons: 91
   dropout_rate: 0.080122423144556634
   activation: sigmoid
   normalize: False
   dropout_rate: 0.013421302851038347
   batch_size: 138
   optimizer: SGD


## Transformer

### Transformer Model implementation

In [24]:
import math

In [25]:
class LearnablePositionEncoding(layers.Layer):
    def __init__(self, d_model, dropout=0.1, max_len=10):
        super(LearnablePositionEncoding, self).__init__()
        self.dropout= layers.Dropout(dropout)
        # self.encoding = tf.Variable(tf.random.uniform((max_len, 1, d_model), -0.2, 0.2), trainable=True)
        self.encoding = tf.Variable(tf.random.uniform((max_len, d_model), -0.2, 0.2), trainable=True)

    def call(self, inputs):
        # print(f"Pos_Encoding Matrix Shape: {self.encoding[:inputs.shape[0], :].shape}")
        x = inputs + self.encoding[:inputs.shape[0], :]
        return self.dropout(x)

In [26]:
class TransformerBatchNormEncoderLayer(layers.Layer):

    def __init__(self, num_heads, d_model, dropout, dim_ff, activation):
        super(TransformerBatchNormEncoderLayer, self).__init__()
        self.self_attn = layers.MultiHeadAttention(num_heads, d_model, dropout=dropout)

        self.linear1 = layers.Dense(dim_ff, activation=activation)
        self.dropout = layers.Dropout(dropout)
        self.linear2 = layers.Dense(d_model, activation='linear')

        self.norm1 = layers.BatchNormalization(epsilon=1e-5)
        self.norm2 = layers.BatchNormalization(epsilon=1e-5)
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)

    def call(self, value):
        mha_out = self.self_attn(value, value)
        drop1_out = self.dropout(mha_out)
        norm1_out = self.norm1(drop1_out)
        combined_1 = norm1_out + drop1_out

        linear1_out = self.linear1(combined_1)
        drop2_out = self.dropout1(linear1_out)
        linear2_out = self.linear2(drop2_out)
        drop3_out = self.dropout2(linear2_out)

        norm2_out = self.norm2(drop3_out)

        out = norm2_out + combined_1

        return out

In [27]:
class TransformerBatchNormEncoderBlock(layers.Layer):

    def __init__(self, num_layers, num_heads, d_model, dropout, dim_ff, activation):
        super(TransformerBatchNormEncoderBlock, self).__init__()
        self.model = keras.Sequential()

        for _ in range(num_layers):
            self.model.add(TransformerBatchNormEncoderLayer(num_heads, d_model, dropout, dim_ff, activation))

    def call(self, inputs):
        return self.model(inputs)

In [28]:
class TransformerEncoderRegressor(layers.Layer):

    def __init__(self, max_len, d_model, n_heads, num_layers, dim_ff, num_classes, dropout=0.1, activation='relu', num_output_layers=1):
        super(TransformerEncoderRegressor, self).__init__()

        self.max_len = max_len
        self.d_model = d_model
        self.n_heads = n_heads
        self.num_classes = num_classes

        self.input_embedding = layers.Dense(d_model, activation='linear')
        self.pos_encoding = LearnablePositionEncoding(d_model, dropout, max_len)

        self.encoder = TransformerBatchNormEncoderBlock(num_layers, n_heads, d_model, dropout, dim_ff, activation)

        self.dropout = layers.Dropout(dropout)

        self.flatten_layer = layers.Flatten()
        self.output_layers = self.build_output_module(d_model, num_output_layers, num_classes, dropout)


    def build_output_module(self, d_model, num_output_layers, num_classes, dropout=0.1):
        output = keras.Sequential()

        i = 1
        while i < num_output_layers:
            output.add(layers.Dense(d_model, activation='relu'))
            output.add(layers.Dropout(dropout))
            i += 1

        output.add(layers.Dense(num_classes, activation='linear'))

        return output

    def call(self, input):
        x = self.input_embedding(input) * math.sqrt(self.d_model)

        # print(f"shape before pos_encoding: {x.shape}") # okay
        x = self.pos_encoding(x) # problem!

        # print(f"shape before transformer: {x.shape}") # not okay
        x = self.encoder(x)
        x = self.dropout(x)

        # x = x.reshape(x.shape[0], -1)
        # print(f"shape before reshape: {x.shape}")
        # x = tf.reshape(x, (x.shape[0], -1))
        # x = tf.reshape(x, (x.shape[0], -1))
        x = self.flatten_layer(x)


        output = self.output_layers(x)

        return output

In [29]:
def get_model(input_shape, max_len, d_model, n_heads, num_layers, dim_ff, num_classes, dropout=0.1, activation='relu', num_output_layers=1):
    input = layers.Input(shape=input_shape)

    # print(f"shape before regressor: {input.shape}")
    regressor = TransformerEncoderRegressor(max_len, d_model, n_heads, num_layers, dim_ff, num_classes, dropout, activation, num_output_layers)

    x = regressor(input)

    return keras.Model(input, x)

### Transformer Study

In [30]:
def create_transformer_dataset(trial):
    N = trial.suggest_int('sequence_length', 8, 25)

    train_samples, train_labels, test_samples, test_labels = get_preprocessed_data(N)

    highest_index_train = int(train_samples.shape[0] * OPTIMIZATION_SPLIT)
    highest_index_test = int(test_samples.shape[0] * OPTIMIZATION_SPLIT)

    return train_samples[:highest_index_train], train_labels[:highest_index_train], test_samples[:highest_index_test], test_labels[:highest_index_test]

In [31]:
def create_transformer_model(trial, train_X):
    number_of_inputs = train_X.shape[1]

    internal_dimension = trial.suggest_int('model_dimension', 20, 256)
    number_of_heads = trial.suggest_int('number_of_attention_heads', 2, 10)
    number_of_encoder_layers = trial.suggest_int('encoder_layers', 1, 10)
    number_of_ff_neurons = trial.suggest_int('ff_neurons', 5, 256)
    number_of_ff_layers = trial.suggest_int('hidden_layers', 1, 10)
    dropout_rate = trial.suggest_float('dropout_rate', 0., 0.5)
    activation_function = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])

    model = get_model([number_of_inputs, 9], number_of_inputs, internal_dimension, number_of_heads, number_of_encoder_layers, number_of_ff_neurons, 2, dropout_rate, activation_function, number_of_ff_layers)

    return model

In [32]:
def transformer_objective(trial):
    train_X, train_Y, test_X, test_Y = create_transformer_dataset(trial)

    model = create_transformer_model(trial, train_X)

    monitor = 'val_loss'

    callbacks = [
        # keras.callbacks.EarlyStopping(patience=10),
        optuna.integration.TFKerasPruningCallback(trial, monitor),
    ]

    # lr = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'rmsprop', 'adagrad', 'adam'])
    batch_size = trial.suggest_int('batch_size', 8, 200)

    model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError(), metrics=['mean_absolute_percentage_error'])

    history = model.fit(train_X, train_Y, batch_size=batch_size, validation_split=0.1, verbose=1, epochs=EPOCHS, callbacks=callbacks)

    eval_results = model.evaluate(test_X, test_Y, batch_size=batch_size)
    print(f'Result: {float(eval_results[0])}')

    return float(eval_results[0])

In [33]:
def make_transformer_study():
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler=optuna.samplers.TPESampler())

    study.optimize(transformer_objective, n_trials=NUMBER_OF_TRIALS)

    # show results
    pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
    complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

    print('Study statistics:')
    print('   Number of finished Trials: ', len(study.trials))
    print('   Number of pruned Trials: ', len(pruned_trials))
    print('   Number of complete Trials: ', len(complete_trials))

    print('Best Trial: ')
    trial = study.best_trial

    print('  MSE: ', trial.value)

    print('   Params: ')
    for key, value in trial.params.items():
        print('   {}: {}'.format(key, value))

In [ ]:
make_transformer_study()

In [36]:
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler=optuna.samplers.TPESampler())

study.optimize(transformer_objective, n_trials=NUMBER_OF_TRIALS)

# show results
pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

print('Study statistics:')
print('   Number of finished Trials: ', len(study.trials))
print('   Number of pruned Trials: ', len(pruned_trials))
print('   Number of complete Trials: ', len(complete_trials))

print('Best Trial: ')
trial = study.best_trial

print('  MSE: ', trial.value)

print('   Params: ')
for key, value in trial.params.items():
    print('   {}: {}'.format(key, value))

[I 2022-06-12 20:45:36,249] A new study created in memory with name: no-name-e95e3cda-dbb1-4b48-9686-bc3193ac48f1


Epoch 1/10
2503/2503 [==============================] - 194s 74ms/step - loss: 0.0465 - mean_absolute_percentage_error: 23756938.0000 - val_loss: 0.0031 - val_mean_absolute_percentage_error: 1547225.1250
Epoch 2/10
2503/2503 [==============================] - 184s 73ms/step - loss: 0.0142 - mean_absolute_percentage_error: 12437446.0000 - val_loss: 0.0031 - val_mean_absolute_percentage_error: 1312282.3750
Epoch 3/10
2503/2503 [==============================] - 185s 74ms/step - loss: 0.0097 - mean_absolute_percentage_error: 9418482.0000 - val_loss: 0.0030 - val_mean_absolute_percentage_error: 1533887.8750
Epoch 4/10
2503/2503 [==============================] - 185s 74ms/step - loss: 0.0077 - mean_absolute_percentage_error: 7765007.0000 - val_loss: 0.0030 - val_mean_absolute_percentage_error: 1332687.7500
Epoch 5/10
2503/2503 [==============================] - 189s 76ms/step - loss: 0.0065 - mean_absolute_percentage_error: 6604262.5000 - val_loss: 0.0030 - val_mean_absolute_percentage_err

[I 2022-06-12 21:19:34,452] Trial 0 finished with value: 0.002883722772821784 and parameters: {'sequence_length': 9, 'model_dimension': 65, 'number_of_attention_heads': 3, 'encoder_layers': 7, 'ff_neurons': 61, 'hidden_layers': 6, 'dropout_rate': 0.1403716054129206, 'activation': 'relu', 'optimizer': 'adagrad', 'batch_size': 133}. Best is trial 0 with value: 0.002883722772821784.


Epoch 1/10
28026/28026 [==============================] - 1326s 47ms/step - loss: nan - mean_absolute_percentage_error: nan - val_loss: nan - val_mean_absolute_percentage_error: nan
Epoch 2/10
28025/28026 [============================>.] - ETA: 0s - loss: nan - mean_absolute_percentage_error: nan

[I 2022-06-12 22:06:45,116] Trial 1 pruned. Trial was pruned at epoch 1.


...

[I 2022-06-16 04:37:12,125] Trial 98 pruned. Trial was pruned at epoch 1.


Epoch 1/10
4023/4023 [==============================] - 887s 217ms/step - loss: 9454.3115 - mean_absolute_percentage_error: 107672664.0000 - val_loss: 0.0029 - val_mean_absolute_percentage_error: 75442.1016
Epoch 2/10
4023/4023 [==============================] - ETA: 0s - loss: 61942.3203 - mean_absolute_percentage_error: 112134104.0000

[I 2022-06-16 05:09:01,383] Trial 99 pruned. Trial was pruned at epoch 1.


Epoch 1/10
4464/4464 [==============================] - 626s 138ms/step - loss: 0.0171 - mean_absolute_percentage_error: 17823826.0000 - val_loss: 0.0028 - val_mean_absolute_percentage_error: 660132.9375
Epoch 2/10
4464/4464 [==============================] - 617s 138ms/step - loss: 0.0058 - mean_absolute_percentage_error: 8668210.0000 - val_loss: 0.0028 - val_mean_absolute_percentage_error: 240771.1875
Epoch 3/10
4464/4464 [==============================] - 617s 138ms/step - loss: 0.0046 - mean_absolute_percentage_error: 6413662.0000 - val_loss: 0.0028 - val_mean_absolute_percentage_error: 73434.9766
Epoch 4/10
4464/4464 [==============================] - 618s 138ms/step - loss: 0.0041 - mean_absolute_percentage_error: 5164399.0000 - val_loss: 0.0028 - val_mean_absolute_percentage_error: 65613.9531
Epoch 5/10
4464/4464 [==============================] - ETA: 0s - loss: 0.0038 - mean_absolute_percentage_error: 4388795.0000

[I 2022-06-16 06:03:06,544] Trial 100 pruned. Trial was pruned at epoch 4.


In [37]:
# show results
pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

print('Study statistics:')
print('   Number of finished Trials: ', len(study.trials))
print('   Number of pruned Trials: ', len(pruned_trials))
print('   Number of complete Trials: ', len(complete_trials))

print('Best Trial: ')
trial = study.best_trial

print('  MSE: ', trial.value)

print('   Params: ')
for key, value in trial.params.items():
    print('   {}: {}'.format(key, value))

Study statistics:
   Number of finished Trials:  100
   Number of pruned Trials:  78
   Number of complete Trials:  22
Best Trial: 
  MSE:  0.0028134321328252554
   Params: 
   sequence_length: 13
   model_dimension: 176
   number_of_attention_heads: 4
   encoder_layers: 10
   ff_neurons: 229
   hidden_layers: 9
   dropout_rate: 0.013421302851038347
   activation: sigmoid
   optimizer: adagrad
   batch_size: 35
